In [ ]:
import gdsfactory as gf
from blocks import doubly_clamped_beam,text_outline

@gf.cell
def test_beam_with_label(width):
    c = gf.Component()
    b1 = c << doubly_clamped_beam(length=30, width=width)
    text = c << text_outline(f't{width*1000:.0f}', size=15, layer='DEEP_ETCH', outline_width=0.7, with_mask=False)
    text.movey(2)
    c.flatten()
    b2 = doubly_clamped_beam(length=30, width=width).copy()
    b2.flatten()
    b2.remove_layers(['DEEP_ETCH'])
    b2.remap_layers({(1,0):(3,6)}, recursive=True)
    b2 = c << b2
    b2.movex(35)
    return c

beam = test_beam_with_label(0.05)
beam


In [ ]:
from blocks import my_coupler, etch_depth_square
from functools import partial
coupler_spec = partial(gf.components.grating_coupler_elliptical, wavelength=1.55,nclad=1,fiber_angle=10, n_periods=60)
gc_spec = partial(my_coupler,coupler=coupler_spec)
@gf.cell
def test_coupler():
    c = gf.Component()
    
    gc = gc_spec().copy()
    c << gc
    c.remap_layers({(2,6): (3,6)}, recursive=True)
    c.flatten()
    return c

c = test_coupler()
c

In [ ]:
import numpy as np
top = gf.Component()
beam_array = gf.Component()
for i, width in enumerate(np.arange(0.05, 0.16 , 0.01)):
    beam = beam_array << test_beam_with_label(width)
    beam.movey(20 * i)
beam_array_1 = top << beam_array
beam_array_2 = top << beam_array
beam_array_2.movex(70)
for i in range(4):
    coupler = top << test_coupler()
    coupler.movey(40 * i+20)
    coupler.movex(150)
(top << etch_depth_square((25,25),layer1='DEEP_ETCH',layer2='DEEP_ETCH')).move((140,180))
top.flatten()
top.remove_layers([(1,0)])

In [ ]:
top.write_gds('gds_files/dose_test.gds',with_metadata=False)